In [352]:
import os
import sys
from pathlib import Path

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score

# Project level imports
sys.path.insert(0, '../lib')
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs

In [353]:
# Setup notebook
nbconfig = Nb.setup_notebook()

Please check output/fbgn2chrom.tsv. If it does not exist, run bin/fbgn2chrom.py
last updated: 2018-09-27 
Git hash: 3e6018d790e56d8ccd70647a7eccff8c120a8e6d


In [354]:
# Connect to data store
store = pd.HDFStore('../output/sra.h5', mode='r')

In [355]:
complete_srx = store['aln/complete'].srx.unique().tolist()

In [356]:
from pymongo import MongoClient
try:
    with open('../output/.mongodb_host', 'r') as fh:
        host = fh.read().strip()
except FileNotFoundError:
    host = 'localhost'

mongoClient = MongoClient(host=host, port=27017)
db = mongoClient['sramongo']
ncbi = db['ncbi']

## Pull Out Library Strategy

In [357]:
libstrat = pd.DataFrame(list(ncbi.aggregate([
    {
        '$match': {
            '_id': {'$in': complete_srx},
        }
    },
    {
        '$unwind': {
            'path': "$runs"
        }
    },
    {
        '$project': {
            '_id': 0,
            'srx': '$srx',
            'srr': '$runs.srr',
            'library_strategy': '$sra.experiment.library_strategy'
        }
    }
]))).set_index(['srx', 'srr'])

libstrat_srx = libstrat.reset_index().groupby('srx').first().library_strategy

In [358]:
srrs = libstrat.index.get_level_values('srr')
srxs = libstrat_srx.index

## Build Feature Table

In [377]:
# collect RNA-seq metrics
cols = [
    'PCT_CODING_BASES',
    'PCT_UTR_BASES',
    'PCT_INTRONIC_BASES',
    'PCT_INTERGENIC_BASES',
    'PCT_MRNA_BASES',
    'MEDIAN_CV_COVERAGE',
    'MEDIAN_5PRIME_BIAS',
    'MEDIAN_3PRIME_BIAS'
]

cm = store.select('prealn/workflow/collectrnaseqmetrics/unstranded', where='srx == srxs', columns=cols)

# collect RNA-seq metrics gene body coverage
gb = store.select('prealn/workflow/collectrnaseqmetrics/genebody', where='srx == srxs')

# markduplicates
cols = [
    'PERCENT_DUPLICATION',
]

mark = store.select('prealn/workflow/markduplicates', where='srx == srxs', columns=cols)

# feature counts summary
cols = [
    'Assigned',
    'Unassigned_Ambiguity',
    'Unassigned_MultiMapping',
    'Unassigned_NoFeatures',
    'Unassigned_Unmapped'
]

feature_summary = store.select('prealn/workflow/feature_counts/summary', columns=cols, where='srx == srxs')

# Munge together
dat_by_srr = cm.join(gb).join(mark).join(feature_summary)
dat_by_srr_no_na = dat_by_srr.fillna(dat_by_srr.mean().to_dict(), axis=0)
dat_by_srr_no_na.shape

# Coverage counts
genic = pd.read_parquet('../output/aln-downstream-wf/aggregate_genic_counts.parquet').reindex(srxs)['count']
genic.name = 'genic'
genic = genic.astype(np.float64)

intergenic = pd.read_parquet('../output/aln-downstream-wf/aggregate_intergenic_counts.parquet').reindex(srxs)['count']
intergenic.name = 'intergenic'
intergenic = intergenic.astype(np.float64)

junctions = pd.read_parquet('../output/aln-downstream-wf/aggregate_junction_counts.parquet').reindex(srxs)['count']
junctions.name = 'junctions'
junctions = junctions.astype(np.float64)

coverage = pd.concat([genic, intergenic, junctions], axis=1)

# Make feature set with all features
features = dat_by_srr_no_na.reset_index().groupby('srx').mean().join(coverage)

In [378]:
# Only look at things we have features for
Y = libstart_srx.reindex(features.index)

# split out the OTHER category
Y_OTHER = Y[Y == 'OTHER'].copy()
features_OTHER = features.reindex(Y_OTHER.index).dropna()

Y = Y[Y != 'OTHER'].copy()
features = features.reindex(Y.index)

In [381]:
# Encode labels
encoder = LabelEncoder()
Y_enc = encoder.fit_transform(Y)

## Random Forest Classifier

First I want to construct a random forest classifier that is trained on a subset of the data and can classify samples. I then look for instances where a sample appears to be mis classified.

There is a small problem with train_test_split, because of differences in class size there are some classes that are missing from the training set. This throws an error when calling precision.

In [425]:
# Make training and testing datasets
X_train, X_test, Y_train, Y_test = train_test_split(features, Y_enc)

# Run a random forest classifier
from sklearn.ensemble import RandomForestClassifier

classifier_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('clf', RandomForestClassifier(n_estimators=100, random_state=42))
])

classifier_pipeline.fit(X_train, Y_train)

# Test
Y_test_pred = classifier_pipeline.predict(X_test)
print(precision_score(Y_test, Y_test_pred, average='weighted'))

0.971993846575


/home/fearjm/miniconda3/envs/ncbi_remap/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Check for miss-classification

In [426]:
miss = pd.DataFrame(dict(annot=encoder.inverse_transform(Y_enc), predicted=encoder.inverse_transform(Y_features)), index=features.index)
random_forest_miss_classified = miss[miss.annot != miss.predicted].copy()

In [427]:
random_forest_miss_classified.query('annot == "RNA-Seq"')

,annot,predicted
srx,,
ERX278731,RNA-Seq,EST
SRX1006593,RNA-Seq,ChIP-Seq


In [428]:
random_forest_miss_classified.query('annot == "ChIP-Seq"')

,annot,predicted
srx,,
SRX046659,ChIP-Seq,WGS
SRX046660,ChIP-Seq,WGS
SRX1056719,ChIP-Seq,WGS
SRX1059356,ChIP-Seq,RNA-Seq
SRX1134694,ChIP-Seq,WGS
SRX1168458,ChIP-Seq,RNA-Seq
SRX2011093,ChIP-Seq,WGS
SRX2068664,ChIP-Seq,WGS
SRX215623,ChIP-Seq,WGS


In [429]:
random_forest_miss_classified.query('annot == "WGS"')

,annot,predicted
srx,,
DRX000998,WGS,ChIP-Seq
DRX001000,WGS,ChIP-Seq
ERX004310,WGS,ChIP-Seq
SRX000529,WGS,ChIP-Seq
SRX000535,WGS,ChIP-Seq
SRX000536,WGS,ChIP-Seq
SRX000558,WGS,ChIP-Seq
SRX006305,WGS,ChIP-Seq
SRX015974,WGS,ChIP-Seq


### Re-classify OTHER

In [430]:
Y_features = classifier_pipeline.predict(features)
Y_OTHER_features = classifier_pipeline.predict(features_OTHER)

In [431]:
_dat = pd.Series(encoder.inverse_transform(Y_OTHER_features), index=features_OTHER.index)
_dat.value_counts()

RNA-Seq      1361
ChIP-Seq      784
WGS           362
EST            45
MNase-Seq      13
miRNA-Seq       3
RIP-Seq         2
CLONEEND        2
dtype: int64

#### Look at MNase-Seq

In [432]:
_dat[_dat == 'MNase-Seq']

srx
ERX365144    MNase-Seq
ERX365194    MNase-Seq
SRX518359    MNase-Seq
SRX518360    MNase-Seq
SRX518366    MNase-Seq
SRX518367    MNase-Seq
SRX518368    MNase-Seq
SRX518372    MNase-Seq
SRX518385    MNase-Seq
SRX518394    MNase-Seq
SRX518395    MNase-Seq
SRX518398    MNase-Seq
SRX518403    MNase-Seq
dtype: object

#### Look at CLONEEND

In [433]:
_dat[_dat == 'CLONEEND']

srx
SRX008031    CLONEEND
SRX008034    CLONEEND
dtype: object

In [435]:
# What is cloneend?
libstart_srx[libstart_srx == 'CLONEEND']

srx
SRX005000    CLONEEND
SRX005001    CLONEEND
SRX005002    CLONEEND
SRX005004    CLONEEND
SRX005008    CLONEEND
SRX005014    CLONEEND
SRX005017    CLONEEND
Name: library_strategy, dtype: object

SRX008031 and SRX008034 are from a small RNA-Seq.
All the annotated CLONEEND are from the sample project SRP000784. They are also small rna (piRNA). 


#### Look at RIP-Seq

In [437]:
_dat[_dat == 'RIP-Seq']

srx
ERX249572    RIP-Seq
SRX017296    RIP-Seq
dtype: object

* ERX249572 is CLIP-Seq
* SRX017296 is related to piRNAs using small RNA purification

#### Look at RNA-Seq

In [438]:
_dat[_dat == 'RNA-Seq'].sample(20)

srx
ERX1495556    RNA-Seq
ERX1495558    RNA-Seq
ERX1475753    RNA-Seq
SRX960112     RNA-Seq
SRX693212     RNA-Seq
SRX1850772    RNA-Seq
ERX1475830    RNA-Seq
SRX145917     RNA-Seq
ERX1475823    RNA-Seq
SRX826291     RNA-Seq
SRX960185     RNA-Seq
SRX1743177    RNA-Seq
SRX2053346    RNA-Seq
SRX625656     RNA-Seq
SRX1868782    RNA-Seq
ERX365299     RNA-Seq
ERX1475792    RNA-Seq
ERX1475875    RNA-Seq
ERX1475838    RNA-Seq
ERX1495719    RNA-Seq
dtype: object

* ERX1495556, ERX1495558: 5' CAGE
* ERX1475753: 3' tag
* SRX960112: mmPCR-seq
* SRX693212: 4c
* SRX1743177: GRO-Seq
* ERX1475838: 3' tag

#### Look at WGS

In [440]:
_dat[_dat == 'WGS'].sample(20)

srx
SRX1048629    WGS
ERX364954     WGS
ERX364789     WGS
ERX149376     WGS
SRX1048635    WGS
ERX365096     WGS
SRX518365     WGS
ERX365342     WGS
ERX365093     WGS
SRX3202392    WGS
ERX364928     WGS
ERX365135     WGS
ERX364913     WGS
SRX141932     WGS
ERX365207     WGS
SRX1048665    WGS
ERX365147     WGS
SRX1048657    WGS
ERX364717     WGS
ERX364873     WGS
dtype: object

looks mostly 4c

In [447]:
# grab all title with -seq
titles = pd.DataFrame(list(ncbi.aggregate([
    {
        '$match': {
            'sra.experiment.title': {'$regex': '\w[-_]seq', '$options': '-i'}
        }
    },
    {
        '$project': {
            '_id': 0,
            'srx': '$srx',
            'title': '$sra.experiment.title'
        }
    },
])))

In [485]:
# titles often has the seq technology in them
import re
regex = re.compile('(?P<tseq>.*[-_]seq)', re.IGNORECASE)
title_seq = titles.title.str.extract(regex)
title_seq.index = titles.srx

In [526]:
res = []
for t in title_seq.tseq.tolist():
    _t = t.lower()
    _t = _t.replace('_seq', '-seq')
    _t = _t.replace('_', ' ')
    res.append(re.findall('\w+-seq', _t)[0])
    
title_seq['tseq'] = res

In [527]:
title_seq.tseq.value_counts()

rna-seq            11724
dna-seq             7036
chip-seq            3603
4c-seq               726
ncrna-seq            513
rad-seq              332
rip-seq              296
mnase-seq            254
tag-seq              245
mirna-seq            121
bisulfite-seq         96
extraction1-seq       72
tada-seq              64
mmpcr-seq             63
pool-seq              61
mrna-seq              57
atac-seq              51
faire-seq             49
start-seq             32
extraction2-seq       24
ka-seq                24
smrna-seq             22
12-seq                22
mtail-seq             21
pro-seq               21
re-seq                20
gro-seq               20
clip-seq              16
wpp-seq               16
input-seq             12
24-seq                12
degradome-seq         11
ribo-seq              11
chirp-seq             10
extraction3-seq        9
icechip-seq            9
sns-seq                9
dnase-seq              6
nascent-seq            6
pb-seq                 6


In [525]:
title_seq.join(libstrat_srx).dropna().query('tseq == "deep-seq"')

,tseq,library_strategy
srx,,
SRX111555,deep-seq,OTHER


## Isolation Forest

In [407]:
from sklearn.ensemble import IsolationForest

### Split data by library stategy

In [252]:
features_rnaseq = features.join(libstrat_srx)\
    .query('library_strategy == "RNA-Seq"')\
    .drop('library_strategy', axis=1)

In [253]:
features_chipseq = features.join(libstrat_srx)\
    .query('library_strategy == "ChIP-Seq"')\
    .drop('library_strategy', axis=1)

In [254]:
features_wgs = features.join(libstrat_srx)\
    .query('library_strategy == "WGS"')\
    .drop('library_strategy', axis=1)

### RNA-Seq with spiked in ChIP-Seq

In [408]:
X_train, X_test = train_test_split(pd.concat([features_rnaseq, features_chipseq.sample(n=200)]))

In [409]:
_cnt1, _cnt2 = X_train.index.isin(features_chipseq.index).sum(), X_test.index.isin(features_chipseq.index).sum()
print(f'There are {_cnt1} ChIP-Seq in Train, and {_cnt2} ChIP-Seq in Test.')

There are 156 ChIP-Seq in Train, and 44 ChIP-Seq in Test.


In [410]:
isolation_pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=12)), 
    ('isolation', IsolationForest(behaviour='new', random_state=42, max_samples='auto', contamination='auto'))
])

isolation_pipeline.fit(X_train)

Pipeline(memory=None,
     steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=12, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('isolation', IsolationForest(behaviour='new', bootstrap=False, contamination='auto',
        max_features=1.0, max_samples='auto', n_estimators=100,
        n_jobs=None, random_state=42, verbose=0))])

In [411]:
print(np.unique(isolation_pipeline.predict(X_test), return_counts=True))

predicted = pd.Series(isolation_pipeline.predict(X_test), index=X_test.index).sort_values()
predicted.name = 'predicted'
_dat = pd.concat([predicted, libstrat_srx], axis=1, sort=True).dropna().query("predicted == -1 & library_strategy == 'ChIP-Seq'")
print(_dat.shape[0])
_dat

(array([-1,  1]), array([ 223, 3433]))
33


,predicted,library_strategy
ERX088873,-1.0,ChIP-Seq
ERX103225,-1.0,ChIP-Seq
ERX103237,-1.0,ChIP-Seq
SRX033319,-1.0,ChIP-Seq
SRX046659,-1.0,ChIP-Seq
SRX062281,-1.0,ChIP-Seq
SRX1056715,-1.0,ChIP-Seq
SRX111780,-1.0,ChIP-Seq
SRX1134695,-1.0,ChIP-Seq
SRX1433390,-1.0,ChIP-Seq


Isolation forest appears to be working okay. Depends on the level of contamination. 

1. run isolation forest on RNA-seq data.
2. Get list of outliers
3. compare to random forest miss-classified

List of 5' and 3' bias can be useful to split these datasets out.